Will solve:

https://github.com/CONABIO/kube_sipecam_playground/issues/14

# Set up minikube and usage of docker image for MAD-Mex + kale in AWS

Will follow: 

* For minikube: [minikube_sipecam/setup](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/setup#aws)

* docker image for MAD-Mex: [kube_sipecam/dockerfiles/MAD_Mex/odc_kale](https://github.com/CONABIO/kube_sipecam/tree/master/dockerfiles/MAD_Mex/odc_kale) and [minikube_sipecam/deployments/MAD_Mex](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/deployments/MAD_Mex/)

* Reference for this nbook: 

[1_issue_5_basic_setup_in_AWS_for_MAD_Mex_classif_pipeline](https://github.com/CONABIO/kube_sipecam_playground/blob/master/MAD_Mex/notebooks/1_issue_5_basic_setup_in_AWS_for_MAD_Mex_classif_pipeline.ipynb)

[1_issue_10_basic_setup_in_AWS_for_MAD_Mex_classif_pipeline](https://github.com/CONABIO/kube_sipecam_playground/blob/master/MAD_Mex/notebooks/2_issues_and_nbooks/1_issue_10_basic_setup_in_AWS_for_MAD_Mex_classif_pipeline.ipynb.ipynb)

Will use [minikube_sipecam/deployments/MAD_Mex/hostpath_pv](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/deployments/MAD_Mex/hostpath_pv)

## Instance

In AWS account we can select ami: `minikube-sipecam` which has next description:

*Based in k8s-1.16-debian-buster-amd64-hvm-ebs-2020-04-27 - ami-0ab39819e336a3f3f Contains kubectl 1.19.1 minikube 1.13.0 kubeflow 1.0.2*

and instance `m5.2xlarge` with `100` gb of disk.

Use next bash script for user data:

```
#!/bin/bash
##variables:
region=us-west-2
name_instance=minikube-10-09-2020
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region
```

**Ssh to instance, all commands will be executed as root**

`sudo su`


**Next will start minikube and kubeflow pods:**

```
cd /root && minikube start --driver=none

cd /opt/kf-test && /root/kfctl apply -V -f kfctl_k8s_istio.v1.0.2.yaml
```


Check pods and status with:

```
minikube status

minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured
```

```
kubectl get pods -n kubeflow

#all running except:
spark-operatorcrd-cleanup-2p7x2                                0/2     Completed   0          7m6s
```



**To access kubeflow UI set:**

```
export INGRESS_HOST=$(minikube ip)
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
echo $INGRESS_PORT
```


**And go to:**

```
http://<ipv4 of ec2 instance>:$INGRESS_PORT
```



## Deployments and services 


**Set:**